In [92]:
from xgboost import plot_importance
from xgboost import XGBClassifier
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

import random

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from statsmodels.api import Logit
from sklearn.metrics import accuracy_score,recall_score,f1_score
from sklearn.metrics import confusion_matrix

from sklearn.tree import export_graphviz
import graphviz
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier

from sklearn.metrics import f1_score, roc_curve, auc
from sklearn.svm import SVC
from sklearn.metrics import f1_score, roc_curve, auc,roc_auc_score
from sklearn.neighbors import KNeighborsClassifier
from imblearn.over_sampling import SMOTE
from sklearn.datasets import make_classification
from sklearn.decomposition import PCA
import scipy.stats
from scipy import stats
%matplotlib inline
matplotlib.rc('font',family = 'Malgun Gothic')

In [93]:
import warnings
warnings.filterwarnings(action='ignore')

In [94]:
import statsmodels.api as sm

In [95]:
df_raw = pd.read_csv("DataSet_final.csv")

In [96]:
df_raw['Target'].mean()

105.77590361445783

In [97]:
# 더미 변수화
df_raw[['Ox_Chamber']]=df_raw[['Ox_Chamber']].astype(str)
df_raw[['photo_soft_Chamber']]=df_raw[['photo_soft_Chamber']].astype(str)
df_raw[['lithography_Chamber']]=df_raw[['lithography_Chamber']].astype(str)
df_raw[['Etching_Chamber']]=df_raw[['Etching_Chamber']].astype(str)
df_raw[['Chamber_Num']]=df_raw[['Chamber_Num']].astype(str)
df_raw[['Wavelength']]=df_raw[['Wavelength']].astype(str)

# wet, dry 구분
df_raw.loc[df_raw['type']=='dry','type']=0
df_raw.loc[df_raw['type']=='wet','type']=1

# Vapor / H2O, O2 구분
df_raw.loc[df_raw['Vapor']=='H2O','Vapor']=0
df_raw.loc[df_raw['Vapor']=='O2','Vapor']=1

# Vapor / H2O, O2 구분
df_raw.loc[df_raw['Vapor']=='H2O','Vapor']=0
df_raw.loc[df_raw['Vapor']=='O2','Vapor']=1

# Wavelength 365, 405, 436
df_raw.loc[df_raw['Wavelength']=='365','Wavelength']=0
df_raw.loc[df_raw['Wavelength']=='405','Wavelength']=1
df_raw.loc[df_raw['Wavelength']=='436','Wavelength']=2

In [98]:
df_list_before=['type', 'Temp_OXid', 'Vapor', 'ppm','Pressure', 'Oxid_time', 'thickness',
        'resist_target', 'N2_HMDS', 'pressure_HMDS', 'temp_HMDS','temp_HMDS_bake', 'time_HMDS_bake', 'spin1', 'spin2', 'spin3','photoresist_bake', 'temp_softbake', 'time_softbake',
        'Line_CD','Wavelength', 'Resolution', 'Energy_Exposure',
        'Thin F4','Thin F2','Thin F3','Thin F1', 'Temp_Etching','Source_Power', 'Selectivity',
        'Flux60s', 'Flux90s', 'Flux160s', 'Flux480s','Flux840s', 'input_Energy', 'Temp_implantation','Furance_Temp', 'RTA_Temp']

In [99]:
# ppm별로 나눌 필요=> dry(O2), wet(H2O) 별로 모델 나눔
df_dummy=pd.get_dummies(df_raw[df_list_before])

In [100]:
# # 불량판단 기준 달라질 것. 이 파라미터에 따른 성능 생각할 필요
# Y_wet.loc[Y_wet.Target<183]=0
# Y_wet.loc[Y_wet.Target>=183]=1
# Y_dry.loc[Y_dry.Target<183]=0
# Y_dry.loc[Y_dry.Target>=183]=1
# len(X),len(Y)
# df_train_x,df_test_x,df_train_y,df_test_y = train_test_split(X,Y,test_size = 0.3,random_state = 1234)

In [101]:
# 데이터 나누기
df_x = df_raw[df_list_before]
df_y = df_raw['Quality']
df_train_x,df_test_x,df_train_y,df_test_y = train_test_split(df_x,df_y,test_size = 0.3,random_state = 1234)


In [102]:
df_test_x_1 = df_test_x.copy()

In [103]:
range_1 = [] #Temp_Oxid
range_2 = [] #Pressure
range_3 = [] #temp_HMDS_bake
range_4 = [] #photoresist_bake
range_5 = [] #spin2
range_6 = [] #Temp_implantation
range_7 = []

In [104]:
for i in range(1, 250):
    range_1.append(random.uniform(1282.139,1310.709))
    range_2.append(random.uniform(0.180,0.193))
    range_3.append(random.uniform(207.765,209.521))
    range_4.append(random.uniform(1282.139,1310.709))
    range_5.append(random.uniform(4105.862,4208.676))
    range_6.append(random.uniform(97.744,100.22))
    range_7.append(random.uniform(13,220))

In [105]:
# Temp_OXid','Pressure','temp_HMDS_bake','photoresist_bake','spin2','Temp_implantation'
df_test_x_1['Temp_OXid']=range_1
df_test_x_1['Pressure']=range_2
df_test_x_1['temp_HMDS_bake']=range_3
df_test_x_1['photoresist_bake']=range_4
df_test_x_1['spin2']=range_5
df_test_x_1['Temp_implantation']=range_6
df_test_x_1['Thin F4']=range_7

In [106]:
len(df_test_x)

249

In [107]:
len(df_test_x_1)

249

In [22]:
smote = SMOTE(random_state=0)

In [113]:
X_train, Y_train = smote.fit_resample(df_train_x, df_train_y)
X_test, Y_test = smote.fit_resample(df_test_x, df_test_y)
X_test_1, Y_test_1 = smote.fit_resample(df_test_x_1, df_test_y)

XGBoost

In [26]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import RobustScaler

In [51]:
model_pipe = Pipeline( [ ('scaler', RobustScaler()),
            ('model', XGBClassifier() )] )
model_pipe.fit(X_train,Y_train)

[18:35:30] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


Pipeline(steps=[('scaler', RobustScaler()),
                ('model',
                 XGBClassifier(base_score=0.5, booster='gbtree',
                               colsample_bylevel=1, colsample_bynode=1,
                               colsample_bytree=1, gamma=0, gpu_id=-1,
                               importance_type='gain',
                               interaction_constraints='',
                               learning_rate=0.300000012, max_delta_step=0,
                               max_depth=6, min_child_weight=1, missing=nan,
                               monotone_constraints='()', n_estimators=100,
                               n_jobs=8, num_parallel_tree=1, random_state=0,
                               reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
                               subsample=1, tree_method='exact',
                               validate_parameters=1, verbosity=None))])

In [28]:
param_grid1 = {'model__booster':['gbtree'],
               'model__nrounds':[90,100],
             'model__max_depth':[5,10,15],
             'model__subsample':[0.6,0.7,0.8],
             'model__min_samples_leaf':[0,5,10,15,20],
             'model__process_type':['default', 'update'],
             'model__objective':['binary:logistic'],
             'model__random_state':[1234]}

# 나 엄연경인데 뭐 불만있나?

In [52]:
# Create a GridSearchCV object
xg_grid = GridSearchCV(estimator= model_pipe,
                       param_grid=param_grid1,
                       scoring='roc_auc',
                       n_jobs=-1,
                       cv=5,
                       refit=True, 
                       return_train_score=True)

xg_grid.fit(X_train, Y_train)

[18:39:17] WARNING: ../src/learner.cc:541: 
Parameters: { min_samples_leaf, nrounds } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[18:39:17] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('scaler', RobustScaler()),
                                       ('model',
                                        XGBClassifier(base_score=0.5,
                                                      booster='gbtree',
                                                      colsample_bylevel=1,
                                                      colsample_bynode=1,
                                                      colsample_bytree=1,
                                                      gamma=0, gpu_id=-1,
                                                      importance_type='gain',
                                                      interaction_constraints='',
                                                      learning_rate=0.300000012,
                                                      max_delta_step=0,
                                                      max_depth=6,
                                              

In [55]:
best_model1 = xg_grid.best_estimator_

In [56]:
best_model1

Pipeline(steps=[('scaler', RobustScaler()),
                ('model',
                 XGBClassifier(base_score=0.5, booster='gbtree',
                               colsample_bylevel=1, colsample_bynode=1,
                               colsample_bytree=1, gamma=0, gpu_id=-1,
                               importance_type='gain',
                               interaction_constraints='',
                               learning_rate=0.300000012, max_delta_step=0,
                               max_depth=15, min_child_weight=1,
                               min_samples_leaf=0, missing=nan,
                               monotone_constraints='()', n_estimators=100,
                               n_jobs=8, nrounds=90, num_parallel_tree=1,
                               process_type='default', random_state=1234,
                               reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
                               subsample=0.6, tree_method='exact',
                               val

In [57]:
xg_grid.best_params_

{'model__booster': 'gbtree',
 'model__max_depth': 15,
 'model__min_samples_leaf': 0,
 'model__nrounds': 90,
 'model__objective': 'binary:logistic',
 'model__process_type': 'default',
 'model__random_state': 1234,
 'model__subsample': 0.6}

In [58]:
y_train_pred1 = best_model1.predict(X_train)
y_test_pred1 = best_model1.predict(X_test)

In [59]:
print("Confusion matrix: \n{}".format(confusion_matrix(Y_test, y_test_pred1)))

Confusion matrix: 
[[210  14]
 [ 23 201]]


In [73]:
df_test_x = pd.read_csv('./test.csv')
df_test_x_1 = pd.read_csv('./test_1.csv')

In [128]:
y_test_pred1 = best_model1.predict(df_test_x_1)

In [129]:
y_test_pred1

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0])

In [134]:
y_test_pred_name = []
for y in y_test_pred1:
    if y == 0:
        y_test_pred_name.append('양품')
    else: y_test_pred_name.append('불량')

In [130]:
np.array(df_test_y)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0])

In [112]:
df_test_y.sum() / len(df_test_y)

0.10040160642570281

In [117]:
y_test_pred2 = best_model1.predict(X_test_1)

In [118]:
y_test_pred2

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [120]:
len(y_test_pred2)

448

In [121]:
Y_test.sum()/ len(Y_test)

0.5

In [74]:
y_test_pred1 = best_model1.predict(df_test_x_1)
# print("Confusion matrix: \n{}".format(confusion_matrix(Y_test, y_test_pred1)))

ValueError: X has 13 features, but RobustScaler is expecting 39 features as input.

In [60]:
from sklearn.metrics import classification_report

In [63]:
print(classification_report(Y_train,y_train_pred1))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       529
           1       1.00      1.00      1.00       529

    accuracy                           1.00      1058
   macro avg       1.00      1.00      1.00      1058
weighted avg       1.00      1.00      1.00      1058



In [64]:
print(classification_report(Y_test,y_test_pred1))

              precision    recall  f1-score   support

           0       0.90      0.94      0.92       224
           1       0.93      0.90      0.92       224

    accuracy                           0.92       448
   macro avg       0.92      0.92      0.92       448
weighted avg       0.92      0.92      0.92       448



In [70]:
print(accuracy_score(Y_test,y_test_pred1))
print(recall_score(Y_test,y_test_pred1))
print(f1_score(Y_test,y_test_pred1))

0.9174107142857143
0.8973214285714286
0.9157175398633257


의사나무결정

In [62]:
model_pipe = Pipeline( [ ('scaler', RobustScaler()),
            ('model', DecisionTreeClassifier(random_state = 1234) )] )
model_pipe.fit(df_train_x,df_train_y)

Pipeline(steps=[('model', DecisionTreeClassifier(random_state=1234))])

In [73]:
para_depth = [depth for depth in range(1, 21)]
para_leaf = [n_leaf * 10 for n_leaf in range(1, 21)]
para_split = [n_split * 20 for n_split in range(1, 21)]
param_grid2 = {'model__max_depth':para_depth,
             'model__min_samples_split':para_split,
             'model__min_samples_leaf':para_leaf}

In [74]:
# Create a GridSearchCV object
df_grid = GridSearchCV(estimator= model_pipe,
                       param_grid=param_grid2,
                       scoring='roc_auc',
                       n_jobs=-1)

df_grid.fit(df_train_x, df_train_y)

GridSearchCV(estimator=Pipeline(steps=[('model',
                                        DecisionTreeClassifier(random_state=1234))]),
             n_jobs=-1,
             param_grid={'model__max_depth': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11,
                                              12, 13, 14, 15, 16, 17, 18, 19,
                                              20],
                         'model__min_samples_leaf': [10, 20, 30, 40, 50, 60, 70,
                                                     80, 90, 100, 110, 120, 130,
                                                     140, 150, 160, 170, 180,
                                                     190, 200],
                         'model__min_samples_split': [20, 40, 60, 80, 100, 120,
                                                      140, 160, 180, 200, 220,
                                                      240, 260, 280, 300, 320,
                                                      340, 360, 380, 400]},
             

In [75]:
best_model2 = df_grid.best_estimator_

In [76]:
best_model2

Pipeline(steps=[('model',
                 DecisionTreeClassifier(max_depth=4, min_samples_leaf=30,
                                        min_samples_split=80,
                                        random_state=1234))])

In [77]:
df_grid.best_params_

{'model__max_depth': 4,
 'model__min_samples_leaf': 30,
 'model__min_samples_split': 80}

In [78]:
y_train_pred2 = best_model2.predict(df_train_x)
y_test_pred2 = best_model2.predict(df_test_x)

In [79]:
print(classification_report(df_train_y,y_train_pred2))

              precision    recall  f1-score   support

           0       0.95      0.99      0.97       529
           1       0.81      0.50      0.62        52

    accuracy                           0.94       581
   macro avg       0.88      0.74      0.79       581
weighted avg       0.94      0.94      0.94       581



In [80]:
print(classification_report(df_test_y,y_test_pred2))

              precision    recall  f1-score   support

           0       0.92      1.00      0.96       224
           1       0.86      0.24      0.38        25

    accuracy                           0.92       249
   macro avg       0.89      0.62      0.67       249
weighted avg       0.92      0.92      0.90       249



In [81]:
print("Confusion matrix: \n{}".format(confusion_matrix(df_test_y, y_test_pred2)))

Confusion matrix: 
[[223   1]
 [ 19   6]]


랜덤포레스트

In [92]:
model_pipe = Pipeline( [ ('scaler', RobustScaler()),
            ('model', RandomForestClassifier(random_state = 1234) )] )
model_pipe.fit(df_train_x,df_train_y)

Pipeline(steps=[('scaler', RobustScaler()),
                ('model', RandomForestClassifier(random_state=1234))])

In [93]:
para_depth = [depth for depth in range(1, 21)]
para_leaf = [n_leaf * 10 for n_leaf in range(1, 21)]
para_split = [n_split * 20 for n_split in range(1, 21)]
para_n_estimator = [n_estimator * 20 for n_estimator in range(1, 101)]
param_grid3 = {"model__max_depth": para_depth,
             "model__min_samples_leaf": para_leaf,
              "model__min_samples_split": para_split,
              "model__n_estimators": para_n_estimator}

In [94]:
estimator = RandomForestClassifier(random_state = 1234)

grid_rf = GridSearchCV(estimator=model_pipe,param_grid = param_grid3, scoring = 'roc_auc',n_jobs = -1)
grid_rf.fit(df_train_x, df_train_y)
print("best estimator model:\n{}".format(grid_rf.best_estimator_))
print("\nbest parametor:\n{}".format(grid_rf.best_params_))
print("\nbest score:\n{}".format(grid_rf.best_score_.round(3)))

KeyboardInterrupt: 

In [ ]:
best_model3 = grid_rf.best_estimator_

In [ ]:
best_model3

In [ ]:
y_train_pred3 = best_model.predict(df_train_x)
y_test_pred3 = best_model.predict(df_test_x)

In [ ]:
print(classification_report(df_train_y,y_train_pred3))

In [ ]:
print(classification_report(df_test_y,y_test_pred3))

그라디언트 부스팅

In [ ]:
# 구하고자 하는 parameter와 범위
param_grid4 = {"model__learning_rate": [0.1,0.2,0.3,0.4,0.5],
              "model__max_depth": [5, 10, 15, 20],
              "model__min_samples_leaf": [10, 20, 30, 40],
              "model__min_samples_split":[10, 20, 30, 40],
              "model__n_estimators":[50,100,150,200]}

In [ ]:
estimator = GradientBoostingClassifier(random_state=1234)

# 정확도가 높은 최적 parameter 찾기
grid_gb = GridSearchCV(estimator, param_grid = param_grid4, scoring="accuracy", n_jobs = -1)
grid_gb.fit(df_train_x, df_train_y)
print("best estimator model: \n{}".format(grid_gb.best_estimator_))
print("\nbest parameter: \n{}".format(grid_gb.best_params_))
print("\nbest score: \n{}".format(grid_gb.best_score_.round(3)))

In [ ]:
best_model4 = grid_gb.best_estimator_

In [ ]:
best_model4

In [ ]:
y_train_pred4 = best_model.predict(df_train_x)
y_test_pred4 = best_model.predict(df_test_x)

In [ ]:
print(classification_report(df_train_y,y_train_pred4))

In [ ]:
print(classification_report(df_test_y,y_test_pred4))